# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

# Load

In [2]:
raw_data = pd.read_json('../data/train.json')
raw_data.head()

,pre_text,post_text,filename,table_ori,table,qa,id,annotation,qa_0,qa_1
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",{'question': 'what was the percentage change i...,Single_JKHY/2009/page_28.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
1,[substantially all of the goodwill and other i...,[the above unaudited pro forma financial infor...,RSG/2008/page_114.pdf,"[[, Year Ended December 31, 2008 (Unaudited), ...","[[, year ended december 31 2008 ( unaudited ),...",{'question': 'what was the percent of the grow...,Single_RSG/2008/page_114.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
2,[in a new business model such as the retail se...,[.],AAPL/2002/page_23.pdf,"[[, 2002, 2001, 2000], [Net sales, $5,742, $5,...","[[, 2002, 2001, 2000], [net sales, $ 5742, $ 5...",{'question': 'what was the percentage change i...,Single_AAPL/2002/page_23.pdf-1,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
3,[( 1 ) includes shares repurchased through our...,[.],UPS/2009/page_33.pdf,"[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...","[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...",{'question': 'what was the difference in perce...,Single_UPS/2009/page_33.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN
4,[( 1 ) includes shares repurchased through our...,[.],UPS/2009/page_33.pdf,"[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...","[[, 12/31/04, 12/31/05, 12/31/06, 12/31/07, 12...",NaN,Double_UPS/2009/page_33.pdf,{'amt_table': '<table class='wikitable'><tr><t...,{'question': 'what is the roi of an investment...,{'question': 'what was the difference in perce...


# Investigation

- Questions are stored under the `.qa` column
- Tables are stored under the `.table` column

- Need to investigate what these are:
    - `.pre_text`
    - `.post_text`
    - `.table_ori`
    - `.annotation`

- Likely not relevant to us:
    - `.filename`
    - `.id`

### Investigating the `.qa` column


In [205]:
sample_question = raw_data.iloc[0]
sample_question

pre_text      [26 | 2009 annual report in fiscal 2008 , reve...
post_text     [year ended june 30 , cash provided by operati...
filename                                  JKHY/2009/page_28.pdf
table_ori     [[, Year ended June 30, 2009], [2008, 2007], [...
table         [[2008, year ended june 30 2009 2008, year end...
qa            {'question': 'what was the percentage change i...
id                               Single_JKHY/2009/page_28.pdf-3
annotation    {'amt_table': '<table class='wikitable'><tr><t...
qa_0                                                        NaN
qa_1                                                        NaN
Name: 0, dtype: object

In [206]:
sample_question.qa

{'question': 'what was the percentage change in the net cash from operating activities from 2008 to 2009',
 'answer': '14.1%',
 'explanation': '',
 'ann_table_rows': [6],
 'ann_text_rows': [],
 'steps': [{'op': 'minus2-1',
   'arg1': '206588',
   'arg2': '181001',
   'res': '25587'},
  {'op': 'divide2-2', 'arg1': '#0', 'arg2': '181001', 'res': '14.1%'}],
 'program': 'subtract(206588, 181001), divide(#0, 181001)',
 'gold_inds': {'table_6': '2008 the net cash from operating activities of year ended june 30 2009 2008 is $ 206588 ; the net cash from operating activities of year ended june 30 2009 2008 is $ 181001 ; the net cash from operating activities of year ended june 30 2009 is $ 174247 ;'},
 'exe_ans': 0.14136,
 'program_re': 'divide(subtract(206588, 181001), 181001)'}

- `pre_text` holds the context before the table appears, `table` and `table_ori` hold the table info, `post_text` holds the context after the table
    - from what I see, `table` is just the lower case version of `table_ori`; no real text cleaning has been performed
- `qa` holds critical information such as
    - `question` and `answer` are self explanatory and easily digested
    - `exe_ans` gives an exact answer in numerical form
- `steps`/`op` appears to be the breakdown of the mathematical operations needed; `program` is the programatical representation of the `steps`/`op`
    - `program_re` combines the steps together into a single operation **This will be most useful**

- `gold_inds` is interesting; *it appears to be the golden extract from the table?*
    - `table_6` hints at the row number
    - the information within the table appears somewhat erroneous, unless I'm misunderstanding it
    - initially thought that my ingestion method caused issues in the column names/headers but I don't think that's the case
    - compare the `sample_question.qa.gold_inds` and my `pd.DataFrame(sample_question.table)`
        - `june 30 2009 2008` continuously appears to be the column name/ header for multiple rows

- Less important
    - `ann_table_rows` appears to show which row is the relevant one within the table (see: `sample_question.table` row 6)

In [207]:
print(sample_question['qa']['gold_inds'])

pd.DataFrame(sample_question.table)

{'table_6': '2008 the net cash from operating activities of year ended june 30 2009 2008 is $ 206588 ; the net cash from operating activities of year ended june 30 2009 2008 is $ 181001 ; the net cash from operating activities of year ended june 30 2009 is $ 174247 ;'}


,0,1,2,3
0,2008,year ended june 30 2009 2008,year ended june 30 2009 2008,year ended june 30 2009
1,net income,$ 103102,$ 104222,$ 104681
2,non-cash expenses,74397,70420,56348
3,change in receivables,21214,-2913 ( 2913 ),-28853 ( 28853 )
4,change in deferred revenue,21943,5100,24576
5,change in other assets and liabilities,-14068 ( 14068 ),4172,17495
6,net cash from operating activities,$ 206588,$ 181001,$ 174247


### Investigating another question

This doesn't appear to be the issue in the second example. Data is not perfect, obviously

In [54]:
sample_question = raw_data.iloc[1]
sample_question

pre_text      [substantially all of the goodwill and other i...
post_text     [the above unaudited pro forma financial infor...
filename                                  RSG/2008/page_114.pdf
table_ori     [[, Year Ended December 31, 2008 (Unaudited), ...
table         [[, year ended december 31 2008 ( unaudited ),...
qa            {'question': 'what was the percent of the grow...
id                               Single_RSG/2008/page_114.pdf-2
annotation    {'amt_table': '<table class='wikitable'><tr><t...
qa_0                                                        NaN
qa_1                                                        NaN
Name: 1, dtype: object

In [55]:
sample_question.qa

{'question': 'what was the percent of the growth in the revenues from 2007 to 2008',
 'answer': '1.3%',
 'explanation': 'the percent growth of the revenue is the difference between the 2 divide by the oldest amount',
 'ann_table_rows': [1],
 'ann_text_rows': [],
 'steps': [{'op': 'minus2-1',
   'arg1': '9362.2',
   'arg2': '9244.9',
   'res': '117.3'},
  {'op': 'divide2-2', 'arg1': '#0', 'arg2': '9244.9', 'res': '1.3%'}],
 'program': 'subtract(9362.2, 9244.9), divide(#0, 9244.9)',
 'gold_inds': {'table_1': 'the revenue of year ended december 31 2008 ( unaudited ) is $ 9362.2 ; the revenue of year ended december 31 2007 ( unaudited ) is $ 9244.9 ;'},
 'exe_ans': 0.012690000000000002,
 'program_re': 'divide(subtract(9362.2, 9244.9), 9244.9)'}

In [208]:
print(sample_question['qa']['gold_inds'])

pd.DataFrame(sample_question.table)

{'table_6': '2008 the net cash from operating activities of year ended june 30 2009 2008 is $ 206588 ; the net cash from operating activities of year ended june 30 2009 2008 is $ 181001 ; the net cash from operating activities of year ended june 30 2009 is $ 174247 ;'}


,0,1,2,3
0,2008,year ended june 30 2009 2008,year ended june 30 2009 2008,year ended june 30 2009
1,net income,$ 103102,$ 104222,$ 104681
2,non-cash expenses,74397,70420,56348
3,change in receivables,21214,-2913 ( 2913 ),-28853 ( 28853 )
4,change in deferred revenue,21943,5100,24576
5,change in other assets and liabilities,-14068 ( 14068 ),4172,17495
6,net cash from operating activities,$ 206588,$ 181001,$ 174247


### Investigating the other columns

- Need to investigate what these are:
    - `.pre_text`
    - `.post_text`
    - `.table_ori`
    - `.annotation`

`.pre_text` appears to be the context before the table; similarly `.post_text` is the text found after the table

We need to combine `pre_text`, the `table`, and then the `post_text` components. They also need to be cleaned appropriately
- trailing whitespaces, need for new-lines etc / general formating for easier digestion into LLM
- incorrect usage of spacing, see: " , " and " ." instead of ", " and "."
- there are cases of artifacts, see : `%%transmsg` at end of `post_text`

In [3]:
sample_question = raw_data.iloc[5]

print(sample_question.pre_text)
print('\n\n')
print(sample_question.post_text)

['tax returns for 2001 and beyond are open for examination under statute .', 'currently , unrecognized tax benefits are not expected to change significantly over the next 12 months .', '19 .', 'stock-based and other management compensation plans in april 2009 , the company approved a global incentive plan which replaces the company 2019s 2004 stock incentive plan .', 'the 2009 global incentive plan ( 201cgip 201d ) enables the compensation committee of the board of directors to award incentive and nonqualified stock options , stock appreciation rights , shares of series a common stock , restricted stock , restricted stock units ( 201crsus 201d ) and incentive bonuses ( which may be paid in cash or stock or a combination thereof ) , any of which may be performance-based , with vesting and other award provisions that provide effective incentive to company employees ( including officers ) , non-management directors and other service providers .', 'under the 2009 gip , the company no longe

### I've found a badly annotated question

The following question seems to be wrong

The question is: *what portion of the total shares subject to outstanding awards is under the 2009 global incentive plan?*

To answer the question we need:

$\frac{\text{outstanding awards in 2009}}{\text{outstanding awards across all years}}$

If I extract the table from

`pd.DataFrame(sample_question.table).to_markdown()`

In [4]:
print(pd.DataFrame(sample_question.table).to_markdown())

|    | 0                          | 1                           | 2                                    |
|---:|:---------------------------|:----------------------------|:-------------------------------------|
|  0 |                            | shares available for awards | shares subject to outstanding awards |
|  1 | 2009 global incentive plan | 2322450                     | 2530454                              |
|  2 | 2004 stock incentive plan  | -                           | 5923147                              |


If for some reason this is wrong, lets extract it from the annotator dict

```
pd.concat(
    pd.read_html(
        StringIO(sample_question.annotation.get('amt_table'))
        )
    )
```

In [5]:
from io import StringIO
pd.concat(
    pd.read_html(
        StringIO(sample_question.annotation.get('amt_table'))
        )
    )

,0,1,2,3
0,1,NaN,shares available for awards,shares subject to outstanding awards
1,2,2009 global incentive plan,2322450,2530454
2,3,2004 stock incentive plan,-,5923147


Both tables indicate the same values/ headers/ column names - so no issue there

Therefore the true answer should be:

2530454 / (5923147 + 2530454)

In [6]:
# My calculation
print(
    (2530454 / (5923147 + 2530454)) * 100
)

# Their calculation
print(
    sample_question.qa.get('answer')
)

29.933444930746077
70.1%


In [ ]:
# See 'table_2' entry: they use numerator to be the value of 2004
# appears to be wrong
sample_question.qa

{'question': 'what portion of the total shares subject to outstanding awards is under the 2009 global incentive plan?',
 'answer': '70.1%',
 'explanation': '',
 'ann_table_rows': [1, 2],
 'ann_text_rows': [],
 'steps': [{'op': 'add2-1',
   'arg1': '2530454',
   'arg2': '5923147',
   'res': '8453601'},
  {'op': 'divide2-2', 'arg1': '5923147', 'arg2': '#0', 'res': '70.1%'}],
 'program': 'add(2530454, 5923147), divide(5923147, #0)',
 'gold_inds': {'table_1': 'the 2009 global incentive plan of shares available for awards is 2322450 ; the 2009 global incentive plan of shares subject to outstanding awards is 2530454 ;',
  'table_2': 'the 2004 stock incentive plan of shares available for awards is - ; the 2004 stock incentive plan of shares subject to outstanding awards is 5923147 ;'},
 'exe_ans': 0.70067,
 'program_re': 'divide(5923147, add(2530454, 5923147))'}

# Checking continual questions

In [357]:
# Hybrid questions are those that have more than one question
# These are found in qa_0 and qa_1 columns
# They do not their qa column populated
# Use this to filter them
simple_questions = raw_data[~raw_data['qa'].isna()]
hybrid_questions = raw_data[raw_data['qa'].isna()]

print(hybrid_questions.iloc[0].qa_0.get('question'))
print(hybrid_questions.iloc[0].qa_1.get('question'))

what is the roi of an investment in ups in 2004 and sold in 2006?
what was the difference in percentage cumulative return on investment for united parcel service inc . compared to the s&p 500 index for the five year period ended 12/31/09?


In [356]:
# Dialogue break
print(hybrid_questions.iloc[0].annotation.get('dialogue_break'))

print('\n\n')

# QA split
print(hybrid_questions.iloc[0].annotation.get('qa_split'))

['what was the fluctuation of the performance price of the ups from 2004 to 2006?', 'and how much does this fluctuation represent in relation to that price in 2004?', 'and from this year to 2009, what was the fluctuation for that stock?', 'what is this fluctuation as a percentage of the 2004 price?', 'and for the s&p 500 index price, what was the fluctuation in those five years?', 'and what percentage does this fluctuation represent in relation to the 2004 price of this stock?', 'what is, then, the difference between the ups percentage and this s&p 500 index one, for this five year period?']



[0, 0, 1, 1, 1, 1, 1]


We need to build a function that is versatile regardless if the entry is a single question (simple) vs multiple (hybrid). We can use `dialogue_break` and `qa_split` which are present for both cases, and use the indeces to split into groups.

This will allow us to construct our prompt correctly.